In [1]:
import os

import numpy as np
import pickle as pkl

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

exec(open("./diagnostics_header.py").read())
import diagnostics_ops as diops
import diagnostics_vars as divars
import diagnostics_vis as divis

integration_x2min = 0. #disk_x2min
integration_x2max = np.pi #disk_x2max


In [ ]:
import sys
if len(sys.argv) < 2: # if run interactively
    supfolder = 'M25'
else: # otherwise use the command line arguments
    supfolder = sys.argv[1]
    if '-navg' in sys.argv: # number of frames for boxcar averaging
        frames_to_avg = int(sys.argv[sys.argv.index('-navg')+1])

In [2]:
# check which machine we're running on, set up threading and path to the data folder(s)
import socket
machine = socket.gethostname()
if machine[:7] == 'perseus':
    machine = 'PERSEUS'
    nproc = 16
    data_pathstem = '/home/ppjanka/tigress_pp/THESIS/%s/' % supfolder
elif machine[:11] == 'tigressdata':
    machine = 'TIGRESSDATA'
    nproc = 4
    data_pathstem = '/home/ppjanka/tigress_pp/THESIS/%s/' % supfolder
elif machine == 'ppjanka-razer':
    machine = 'RAZER'
    nproc = 2
    data_pathstem = '/DATA/Dropbox/LOOTRPV/Princeton_PhD/Thesis/globAccDisk/athena/bin-mhd/%s/' % supfolder
    
print('Running on %s.' % (machine,))

# what folders within data_pathstem store the athdf files
data_folders = ['R4','R5']

# ----------
import glob

# find all the athdf files
athdf_files = []
for data_folder in data_folders:
    athdf_files += glob.glob(data_pathstem + data_folder + '/*out2*.athdf')
athdf_files = sorted(athdf_files)

Running on RAZER.


In [3]:
if False:
    # setup the figure
    fig = plt.figure(figsize=(10,15))
    gs = gridspec.GridSpec(3,1)

# prepare the dictionary to be pickled for reuse
output = {}

# open the last run's data if available
old_data = None
if os.path.isfile(data_pathstem + 'TSeries.pkl'):
    with open(data_pathstem + 'TSeries.pkl', 'rb') as f:
        old_data = pkl.load(f)

In [4]:
if True:
    # Mdot(t)
    vars_obj = divars.Mdot()
    #ax = plt.subplot(gs[0,0])
    output['mdot'] = {}
    for radius in [0.5, 0.06, 0.08, 0.1, 0.12, 0.15, 0.18, 0.21, 0.24, 0.27, 0.3]:
        if old_data != None:
            old_data_here = old_data['mdot'][radius]
        else:
            old_data_here = None
        ops_obj = diops.TSeries_SphSlice(vars_object=vars_obj, r=radius, x2min=integration_x2min, x2max=integration_x2max, type='sum')
        ops_obj.read(athdf_files, old_data=old_data_here)
        output['mdot'][radius] = ops_obj.data
        #_ = ops_obj.plot(fig, ax=ax, log_scale=True, title='auto', kwargs={'label':('r = %.2f sim.u.' % radius)}, vmin=1.0e-5, boxcar_smooth=frames_to_avg)
        with open(data_pathstem + 'TSeries.pkl', 'wb') as f:
            pkl.dump(output, f)
        del ops_obj
    #ax.legend()
    del vars_obj

100%|██████████| 47/47 [00:04<00:00,  9.06it/s]


In [ ]:
# prepare the dictionary to be pickled for reuse
output = {}

# open the last run's data if available
old_data = None
if os.path.isfile(data_pathstem + 'TSeries_noFloor.pkl'):
    with open(data_pathstem + 'TSeries_noFloor.pkl', 'rb') as f:
        old_data = pkl.load(f)

In [ ]:
if False:
    # Mdot(t) ignoring the floor
    vars_obj = divars.Mdot(rho_crit=0.01, vel_crit=-0.4, ignore_floor=True)
    #ax = plt.subplot(gs[0,0])
    output['mdot'] = {}
    for radius in [0.5, 0.06, 0.08, 0.1, 0.125, 0.15, 0.175, 0.2, 0.3, 0.4]:
        if old_data != None:
            old_data_here = old_data['mdot'][radius]
        else:
            old_data_here = None
        ops_obj = diops.TSeries_SphSlice(vars_object=vars_obj, r=radius, x2min=integration_x2min, x2max=integration_x2max, type='sum')
        ops_obj.read(athdf_files, old_data=old_data_here)
        output['mdot'][radius] = ops_obj.data
        #_ = ops_obj.plot(fig, ax=ax, log_scale=True, title='auto', kwargs={'label':('r = %.2f sim.u.' % radius)}, vmin=1.0e-5, boxcar_smooth=frames_to_avg)
        del ops_obj
        with open(data_pathstem + 'TSeries_noFloor.pkl', 'wb') as f:
            pkl.dump(output, f)
    #ax.legend()
    del vars_obj

In [5]:
if False:
    # Reynolds stress (t)
    vars_obj = divars.StressReynolds()
    ax = plt.subplot(gs[1,0])
    ops_obj = diops.TSeries_Total(vars_object=vars_obj, x2min=integration_x2min, x2max=integration_x2max)
    if old_data != None:
        old_data_here = old_data['stress_Reynolds']
    else:
        old_data_here = None
    ops_obj.read(athdf_files, old_data=old_data_here)
    output['stress_Reynolds'] = ops_obj.data
    _ = ops_obj.plot(fig, ax=ax, log_scale=False, title='auto')
    del ops_obj, vars_obj

100%|██████████| 47/47 [01:25<00:00,  2.12s/it]


In [6]:
if False:
    # Maxwell stress (t)
    vars_obj = divars.StressMaxwell()
    ax = plt.subplot(gs[2,0])
    ops_obj = diops.TSeries_Total(vars_object=vars_obj, x2min=integration_x2min, x2max=integration_x2max)
    if old_data != None:
        old_data_here = old_data['stress_Maxwell']
    else:
        old_data_here = None
    ops_obj.read(athdf_files, old_data=old_data_here)
    output['stress_Maxwell'] = ops_obj.data
    _ = ops_obj.plot(fig, ax=ax, log_scale=False, title='auto')
    del ops_obj, vars_obj

100%|██████████| 47/47 [00:58<00:00,  1.44s/it]
